# HW4 - Great Gatsby Sequential Model

1. Choose a book or other text to train your model on (I suggest Project Gutenberg to find .txt files but you can find them elsewhere). Make sure your file is a .txt file. Clean your data (You may use this file we talked about in class as a baseline). Build a sequential model (LSTM, GRU, SimpleRNN, or Transformer) that generates new lines based on the training data (NO TRANSFER LEARNING).

Print out or write 10 generated sequences from your model (Similar to Classwork 17 where we generated new Pride and Prejudice lines, but now with words instead of charachters. Feel free to use this as a reference). Assess in detail how good they are, what they're good at, what they struggle to do well. INCLUDE THESE 10 sequences in your report.

2. Make a new model with ONE substantial adjustment (e.g. use a custom embedding layer if you didn't already, use a pre-trained embedding layer if you didn't already, use a DEEP LSTM/GRU with multiple recurrent layers, use a pre-trained model to do transfer learning and fine-tune it...etc.).

Print out or write 10 generated sequences from your model (Similar to Classwork 17 where we generated new Pride and Prejudice lines, but now with words instead of charachters. Feel free to use this as a reference). INCLUDE THESE 10 sequences in your report. Assess in detail how good they are, what they're good at, what they struggle to do well. Did the performance of your model change?

3. Then create a **technical report** discussing your model building process, the results, and your reflection on it. The report should follow the format in the example including an Introduction, Analysis, Methods, Results, and Reflection section.

In [1]:
# imports
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Input
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.text import Tokenizer

import string

# changeable params
my_file = "TheGreatGatsby.txt"
seq_len = 100

# load doc into memory
def load_doc(filename):
 # open the file as read only
 file = open(filename, 'r')
 # read all text
 text = file.read()
 # close the file
 file.close()
 return text

# turn a doc into clean tokens
def clean_doc(doc):
 # replace '--' with a space ' '
 doc = doc.replace('--', ' ')
 # split into tokens by white space
 tokens = doc.split()
 # remove punctuation from each token
 table = str.maketrans('', '', string.punctuation)
 tokens = [w.translate(table) for w in tokens]
 # remove remaining tokens that are not alphabetic
 tokens = [word for word in tokens if word.isalpha()]
 # make lower case
 tokens = [word.lower() for word in tokens]
 return tokens

# save tokens to file, one dialog per line
def save_doc(lines, filename):
 data = '\n'.join(lines)
 file = open(filename, 'w')
 file.write(data)
 file.close()

# load document
doc = load_doc(my_file)
print(doc[:200])

# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

# organize into sequences of tokens
length = seq_len + 1
sequences = list()
for i in range(length, len(tokens)):
 # select sequence of tokens
 seq = tokens[i-length:i]
 # convert into a line
 line = ' '.join(seq)
 # store
 sequences.append(line)
print('Total Sequences: %d' % len(sequences))

# save sequences to file
out_filename = my_file[:-4] + '_seq.txt'
save_doc(sequences, out_filename)

The Project Gutenberg eBook of The Great Gatsby
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
what
['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'great', 'gatsby', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'united', 'states', 'and', 'most', 'other', 'parts', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', 'you', 'may', 'copy', 'it', 'give', 'it', 'away', 'or', 'reuse', 'it', 'under', 'the', 'terms', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'wwwgutenbergorg', 'if', 'you', 'are', 'not', 'located', 'in', 'the', 'united', 'states', 'you', 'will', 'have', 'to', 'check', 'the', 'laws', 'of', 'the', 'country', 'where', 'you', 'are', 'located', 'before', 'using', 'this', 'ebook', 'title', 'the', 'great', 'gatsby', 'author', 

In [2]:
# load doc into memory
def load_doc(filename):
 # open the file as read only
 file = open(filename, 'r')
 # read all text
 text = file.read()
 # close the file
 file.close()
 return text

# load
doc = load_doc(out_filename)
lines = doc.split('\n')

# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [3]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1


# separate into input and output
sequences = np.array(sequences)
sequences.shape
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

p_train = 0.8

n_train = int(X.shape[0]//(1/p_train))
X_train = X[0:n_train]
y_train = y[0:n_train]
X_test = X[n_train:]
y_test = y[n_train:]



In [9]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 25, input_length=seq_length))
model.add(LSTM(150, return_sequences=True))
model.add(LSTM(150))
model.add(Dense(150, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X_train, y_train, batch_size=128, epochs=225)



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 25)           149925    
                                                                 
 lstm_2 (LSTM)               (None, 100, 150)          105600    
                                                                 
 lstm_3 (LSTM)               (None, 150)               180600    
                                                                 
 dense_2 (Dense)             (None, 150)               22650     
                                                                 
 dense_3 (Dense)             (None, 5997)              905547    
                                                                 
Total params: 1364322 (5.20 MB)
Trainable params: 1364322 (5.20 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/225

In [10]:
from random import randint
from keras.preprocessing.sequence import pad_sequences


def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
  result = list()
  in_text = seed_text
# generate a fixed number of words
  for _ in range(n_words):
# encode the text as integer
    encoded = tokenizer.texts_to_sequences([in_text])[0]
# truncate sequences to a fixed length
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
# predict probabilities for each word
    yhat = np.argmax(model.predict(encoded, verbose=0), axis=-1)
    print(yhat)
# map predicted word index to word
    out_word = ''
    for word, index in tokenizer.word_index.items():
      if index == yhat:
        out_word = word
        break
# append to input
    in_text += ' ' + out_word
    result.append(out_word)
  return ' '.join(result)

# load cleaned text sequences
in_filename = 'TheGreatGatsby_seq.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1


# Generate and assess 10 sequences
for _ in range(10):
    seed_text = lines[randint(0, len(lines))]
    generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
    #print(f"Seed Text: {seed_text}\nGenerated Sequence: {generated}\n")
    print(f"\nGenerated Sequence: {generated}\n")
    #print("Basic Assessment: [Your assessment here]\n")



[123]
[200]
[4]
[189]
[2]
[6]
[892]
[15]
[11]
[6]
[26]
[448]
[8]
[17]
[1]
[2002]
[24]
[11]
[69]
[50]
[855]
[158]
[1]
[614]
[243]
[13]
[3]
[320]
[4]
[2108]
[203]
[14]
[3]
[58]
[237]
[6]
[17]
[1]
[849]
[4]
[488]
[71]
[7]
[1018]
[1250]
[24]
[1]
[515]
[13]
[80]

Generated Sequence: only thing of years and i guessed at his i said someone he had the decency from his eyes then looks under the kitchen steps with a copy of simon called you a little later i had the bottle of second now in constant demand from the heat with them

[15]
[21]
[13]
[1]
[5079]
[270]
[562]
[217]
[16]
[950]
[21]
[2]
[8]
[5284]
[208]
[7]
[21]
[40]
[6]
[364]
[13]
[3]
[535]
[2513]
[1]
[54]
[436]
[319]
[1]
[860]
[4]
[3]
[65]
[103]
[1343]
[5]
[1021]
[53]
[5]
[25]
[118]
[2]
[40]
[6]
[9]
[3]
[26]
[111]
[342]
[51]

Generated Sequence: at me with the easygoing blue coupé gave her lying me and he winced himself in me if i live with a low vulgar the is cars behind the confusion of a man could store to walk down to my car and if i 